In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, parent_dir)

parent_dir = os.path.abspath(os.path.join(os.getcwd().split('render')[0],'gaussian_data'))
sys.path.insert(0, parent_dir)

from gaussian_data.Camera import Camera
from gaussian_data.Frame import Frame
import gaussian_data.Plotters
import gaussian_data.Utils as Utils
from GaussianSplat import GaussianSplat
import numpy as np
from plyfile import PlyData
import numpy as np
import argparse
from io import BytesIO
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import scipy.io
import pandas as pd
from Render import Render

%matplotlib qt

# load hull
path = 'I:/My Drive/Research/gs_data/mov19_2022_03_03/'
real_coord = scipy.io.loadmat(f'{path}/3d_pts/real_coord.mat')['all_coords']
points_3d = {body_wing : pd.DataFrame(Utils.load_hull(body_wing,path),columns = ['X','Y','Z','frame']) for body_wing in ['body','rwing','lwing']}
# initilize objects
frames = [1483]

image_names,points_in_idx = Utils.define_frames(frames,points_3d)
cameras = {f'cam{cam + 1}':Camera(path,cam) for cam in range(4)}
frames = {f'{im_name}.jpg':Frame(path,im_name,points_in_idx[im_name.split('CAM')[0]],real_coord,idx) for idx,im_name in enumerate(image_names)}
# map 3d voxels to 2d pixels
[frames[im_name].map_3d_2d(croped_image = True) for im_name in frames.keys()]
voxel_dict,colors_dict = Utils.get_dict_for_points3d(frames)


In [2]:
def build_rotation(r):
    norm = np.sqrt(r[:,0]*r[:,0] + r[:,1]*r[:,1] + r[:,2]*r[:,2] + r[:,3]*r[:,3])

    q = r / norm[:, None]

    R = np.zeros((q.shape[0], 3, 3))

    r = q[:, 0]
    x = q[:, 1]
    y = q[:, 2]
    z = q[:, 3]

    R[:, 0, 0] = 1 - 2 * (y*y + z*z)
    R[:, 0, 1] = 2 * (x*y - r*z)
    R[:, 0, 2] = 2 * (x*z + r*y)
    R[:, 1, 0] = 2 * (x*y + r*z)
    R[:, 1, 1] = 1 - 2 * (x*x + z*z)
    R[:, 1, 2] = 2 * (y*z - r*x)
    R[:, 2, 0] = 2 * (x*z - r*y)
    R[:, 2, 1] = 2 * (y*z + r*x)
    R[:, 2, 2] = 1 - 2 * (x*x + y*y)
    return R


def build_scaling_rotation(s, r):
    L = np.zeros((s.shape[0], 3, 3))
    R = build_rotation(r)

    L[:,0,0] = s[:,0]
    L[:,1,1] = s[:,1]
    L[:,2,2] = s[:,2]

    L = R @ L
    return L

In [3]:

def homogeneous(points):
    """
    homogeneous points
    :param points: [..., 3]
    """
    return np.column_stack((points, np.ones(points.shape[0])))

def homogeneous_vec(vec, vectoadd = [0,0]):
    """
    homogeneous points
    :param points: [..., 3]
    """
    return np.concatenate((vec,np.tile(np.array([vectoadd]).T,(vec.shape[0],1,1))),axis = 2)


In [4]:

def getProjectionMatrix(znear, zfar, fovX, fovY):
    import math
    tanHalfFovY = math.tan((fovY / 2))
    tanHalfFovX = math.tan((fovX / 2))

    top = tanHalfFovY * znear
    bottom = -top
    right = tanHalfFovX * znear
    left = -right

    P = np.zeros((4, 4))

    z_sign = 1.0

    P[0, 0] = 2.0 * znear / (right - left)
    P[1, 1] = 2.0 * znear / (top - bottom)
    P[0, 2] = (right + left) / (right - left)
    P[1, 2] = (top + bottom) / (top - bottom)
    P[3, 2] = z_sign
    P[2, 2] = z_sign * zfar / (zfar - znear)
    P[2, 3] = -(zfar * znear) / (zfar - znear)
    return P


def focal2fov(focal, pixels):
    import math
    return 2*math.atan(pixels/(2*focal))


def get_inputs(num_points=8):
    length = 0.5
    num_points = 8
    x = np.linspace(-1, 1, num_points) * length
    y = np.linspace(-1, 1, num_points) * length
    x, y = np.meshgrid(x, y)
    means3D = np.stack([x,y, 0 * np.random.rand(*x.shape)], axis=-1).reshape(-1,3)
    quats = np.tile(np.zeros((1,4)),(len(means3D), 1))
    quats[..., 0] = 1.
    scale = length /(num_points-1)
    scales = np.tile(np.ones((1,3)),(len(means3D), 1))*scale
    return means3D, scales, quats

def get_cameras():    
    intrins = np.array([[711.1111,   0.0000, 256.0000,   0.0000],
               [  0.0000, 711.1111, 256.0000,   0.0000],
               [  0.0000,   0.0000,   1.0000,   0.0000],
               [  0.0000,   0.0000,   0.0000,   1.0000]])
    c2w = np.array([[-8.6086e-01,  3.7950e-01, -3.3896e-01,  6.7791e-01],
         [ 5.0884e-01,  6.4205e-01, -5.7346e-01,  1.1469e+00],
         [ 1.0934e-08, -6.6614e-01, -7.4583e-01,  1.4917e+00],
         [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])

    width, height = 512, 512
    focal_x, focal_y = intrins[0, 0], intrins[1, 1]
    viewmat = np.linalg.inv(c2w).T
    FoVx = focal2fov(focal_x, width)
    FoVy = focal2fov(focal_y, height)
    projmat = getProjectionMatrix(znear=0.2, zfar=1000, fovX=FoVx, fovY=FoVy).T
    projmat = viewmat @ projmat
    return intrins, viewmat, projmat, height, width


# Surface splatting (2D Gaussian Splatting)
def setup(means3D, scales, quats, opacities, colors, viewmat, projmat):
    rotations = build_scaling_rotation(scales, quats).permute(0,2,1)

    # 1. Viewing transform
    # Eq.4 and Eq.5
    p_view = (means3D @ viewmat[:3,:3]) + viewmat[-1:,:3]
    uv_view = (rotations @ viewmat[:3,:3])
    M = torch.cat([homogeneous_vec(uv_view[:,:2,:]), homogeneous(p_view.unsqueeze(1))], dim=1)

    T = M @ projmat # T stands for (WH)^T in Eq.9 
    # 2. Compute AABB
    # Homogneous plane is very useful for both ray-splat intersection and bounding box computation
    # we know how to compute u,v given x,y homogeneous plane already; computing AABB is done by a reverse process.
    # i.e compute the x, y s.t. \|hu^4\| = 1 and \|h_v^4\|=1 (distance of gaussian center to plane in the uv space)
    temp_point = torch.tensor([[1.,1., -1.]]).cuda()
    distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
    f = (1 / distance) * temp_point
    point_image = torch.cat(
        [(f * T[..., 0] * T[...,3]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,3]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,3]).sum(dim=-1, keepdims=True)], dim=-1)

    half_extend = point_image * point_image - torch.cat(
        [(f * T[..., 0] * T[...,0]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,1]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,2]).sum(dim=-1, keepdims=True)], dim=-1)

    radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
    center = point_image

    # 3. Perform Sorting
    depth = p_view[..., 2] # depth is used only for sorting
    index = depth.sort()[1]
    T = T[index]
    colors = colors[index]
    center = center[index]
    depth = depth[index]
    radii = radii[index]
    opacities = opacities[index]
    return T, colors, opacities, center, depth, radii


In [5]:
# Make inputs
import matplotlib
num_points1=8
means3d, scales, quats = get_inputs(num_points=num_points1)
intrins, viewmat, projmat, height, width = get_cameras()


intrins = intrins[:3,:3]
colors = matplotlib.colormaps['Accent'](np.random.randint(1,64, 64)/64)[..., :3]

opacities = np.ones(means3d[:,:1].shape)


rotations = build_scaling_rotation(scales,quats) # sutu, svtv (scale * axis direction in gaussian FoR)
# projmat = frames['P1483CAM1.jpg'].projection


# 1. Viewing transform
# Eq.4 and Eq.5
p_view = (means3d @ viewmat[:3,:3]) + viewmat[-1:,:3] # rotate the gaussian mean to camera FoR
uv_view = (rotations @ viewmat[:3,:3]) # rotate to camera FoR

# M is H matrix that representes the transformation from tangent plane to camera. 
# its the scaled axes concatenated to the gaussian mean location - represented in homogeneous coordinates

# !! need to check that the order of axes ar ok for M !!
M = np.concatenate((homogeneous_vec(uv_view[:,:2,:]),homogeneous(p_view)[:,np.newaxis]),axis = 1)
T = M @ projmat # T stands for (WH)^T in Eq.9 - projmat transforms from camera to NDC (screen coordinates)
# T is the transformation of every gaussian from tangent plane to NDC, its homogebnus coordinates. with the rotation matrix 
# representing the axes and the translation vector representing the location of the center of each gaussian. 
# We notice that projmat is a prespective projection matrix. 

# Next, We calculate the radius of the gaussian. We normalize by w to get homogeneus coordinates. In addition we flip Z axis (not sure why) 
# we calculate the distance from the camera to the gaussian mean (this is w, the last row of a homogenues coordinate, deviding by it will give perspective view)
# Notice that the rotation is scaled (in build_scaling_rotation) and is not normalized.

# point_image - the projectes mean of the gaussian (with flipped z)
# half_extend - used to calculate the radius of the gaussian, we take 3 sigma. because the ratation is scaled 
# we calculate the distance for each axis and can get the 3 sigma by multiplying each distance. (we also devide by w to get the prespective view)

temp_point = np.tile([1,1,-1],(T.shape[0],1))
distance  = np.sum(temp_point*T[..., 3] * T[..., 3],-1)
f = (1 / distance[:,np.newaxis]) * temp_point

# distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
point_image = np.column_stack((np.sum(f * T[..., 0] * T[...,3],1),np.sum(f * T[..., 1] * T[...,3],1),np.sum(f * T[..., 2] * T[...,3],1)))

axes_dist = np.column_stack((np.sum(f * T[..., 0] * T[...,0],1),np.sum(f * T[..., 1] * T[...,1],1),np.sum(f * T[..., 2] * T[...,2],1)))

half_extend = point_image * point_image - axes_dist
# distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
# point_image = np.column_stack((np.sum(f * T[..., 0] * T[...,3],1),np.sum(f * T[..., 1] * T[...,3],1),np.sum(f * T[..., 2] * T[...,3],1)))

    # half_extend = point_image * point_image - torch.cat(
    #     [(f * T[..., 0] * T[...,0]).sum(dim=-1, keepdims=True),
    #     (f * T[..., 1] * T[...,1]).sum(dim=-1, keepdims=True),
    #     (f * T[..., 2] * T[...,2]).sum(dim=-1, keepdims=True)], dim=-1)

    # radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
    # center = point_image

# T, colors, opacities, center, depth, radii = setup(means3D, scales, quats, opacities, colors, viewmat, projmat)


In [7]:
import numpy as np
input_file = "I:/My Drive/Research/gs_data/mov19_2022_03_03/2dgs_output/84444b22-6/point_cloud/iteration_1000/point_cloud.ply"
vertices = PlyData.read(input_file)["vertex"]


means3d = np.column_stack((vertices['x'],vertices['y'],vertices['z']))
viewmat = frames['P1483CAM1.jpg'].world_to_cam.T


r = np.column_stack([(vertices[f'rot_{idx}']) for idx in range(4)])
s = np.column_stack(([(vertices[f'scale_{idx}']) for idx in range(2)]))
s = np.column_stack((s,vertices['scale_0']))
rotations = build_scaling_rotation(s,r) # sutu, svtv (scale * axis direction in gaussian FoR)
projmat = frames['P1483CAM1.jpg'].full_proj_transform
intrins = frames['P1483CAM1.jpg'].K_crop


# 1. Viewing transform
# Eq.4 and Eq.5
p_view = (means3d @ viewmat[:3,:3]) + viewmat[-1:,:3] # rotate the gaussian mean to camera FoR
uv_view = (rotations @ viewmat[:3,:3]) # rotate to camera FoR

# M is H matrix that representes the transformation from tangent plane to camera. 
# its the scaled axes concatenated to the gaussian mean location - represented in homogeneous coordinates

# !! need to check that the order of axes ar ok for M !!
M = np.concatenate((homogeneous_vec(uv_view[:,:2,:]),homogeneous(p_view)[:,np.newaxis]),axis = 1)
T = M @ projmat # T stands for (WH)^T in Eq.9 - projmat transforms from camera to NDC (screen coordinates)
# T is the transformation of every gaussian from tangent plane to NDC, its homogebnus coordinates. with the rotation matrix 
# representing the axes and the translation vector representing the location of the center of each gaussian. 
# We notice that projmat is a prespective projection matrix. 

# Next, We calculate the radius of the gaussian. We normalize by w to get homogeneus coordinates. In addition we flip Z axis (not sure why) 
# we calculate the distance from the camera to the gaussian mean (this is w, the last row of a homogenues coordinate, deviding by it will give perspective view)
# Notice that the rotation is scaled (in build_scaling_rotation) and is not normalized.

# point_image - the projectes mean of the gaussian (with flipped z)
# half_extend - used to calculate the radius of the gaussian, we take 3 sigma. because the ratation is scaled 
# we calculate the distance for each axis and can get the 3 sigma by multiplying each distance. (we also devide by w to get the prespective view)

temp_point = np.tile([1,1,-1],(T.shape[0],1))
distance  = np.sum(temp_point*T[..., 3] * T[..., 3],-1)
f = (1 / distance[:,np.newaxis]) * temp_point

# distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
point_image = np.column_stack((np.sum(f * T[..., 0] * T[...,3],1),np.sum(f * T[..., 1] * T[...,3],1),np.sum(f * T[..., 2] * T[...,3],1)))

axes_dist = np.column_stack((np.sum(f * T[..., 0] * T[...,0],1),np.sum(f * T[..., 1] * T[...,1],1),np.sum(f * T[..., 2] * T[...,2],1)))

half_extend = point_image * point_image - axes_dist
radii = np.sqrt(np.maximum(half_extend, 1e-4)) * 3
center = point_image

# 3. Perform Sorting
depth = p_view[..., 2] # depth is used only for sorting
index = np.argsort(depth)
T = T[index]
# colors = colors[index]
center = center[index]
depth = depth[index]
radii = radii[index]
# opacities = opacities[index]


In [56]:
frames['P1483CAM1.jpg'].croped_image.size

(160, 160)

In [52]:
# Rasterization
# 1. Generate pixels
W, H = 160,160
pix_x, pix_y = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
pix = np.stack([pix_x, pix_y], axis=-1)

# 2. Compute ray splat intersection # Eq.9 and Eq.10
pix_flat = pix.reshape(-1, 1, 2)
x = pix_flat[..., :1]
y = pix_flat[..., 1:]
k = -T[None, ..., 0] + x * T[None, ..., 3]
# l = -T[None, ..., 1] + y * T[None, ..., 3]

MemoryError: Unable to allocate 3.48 GiB for an array with shape (25600, 6079, 3) and data type float64

In [100]:
def intersection_point(pixel,T):
    k = -T[..., 0] + pixel[0]*T[...,3]
    l = -T[..., 1] + pixel[1] * T[..., 3]
    points = np.cross(k, l, axis=-1)
    return points[..., :2] / points[..., -1:]

intersection_point([0,10],T)

array([[-0.03984097,  0.04359713],
       [-0.04031687,  0.04156441],
       [-0.04277372,  0.0429387 ],
       ...,
       [-0.06775295, -0.03822734],
       [-0.07266341, -0.03957052],
       [-0.07130239, -0.04003944]])

In [61]:
def intersection_point(pixel,T):
    k = -T[..., 0] + pixel[0]*T[...,3]
    l = -T[..., 1] + pixel[1] * T[..., 3]
    points = np.cross(k, l, axis=-1)
    return points[..., :2] / points[..., -1:]


# Rasterization
# 1. Generate pixels
W, H = int(intrins[0, -1] * 2), int(intrins[1, -1] * 2)
pix_x, pix_y = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
pix = np.stack([pix_x, pix_y], axis=-1)

# 2. Compute ray splat intersection # Eq.9 and Eq.10
pix_flat = pix.reshape(-1, 1, 2)
pixels = np.squeeze(pix_flat)[0:100]
x,y = pixels[:,0],pixels[:,1]
s = np.vstack([intersection_point(pixel,T) for pixel in pixels])

# 3. Add 
# -pass filter # Eq.11
# When a point (2D Gaussian) is viewed from a far distance or at a slanted angle,
# the 2D Gaussian falls between pixels, and no fragment is used to rasterize the Gaussian.
# Add a low-pass filter to handle aliasing.
dist3d = np.sum(s * s, axis=-1)
filtersze = np.sqrt(2) / 2
dist_xycenter = np.hstack([pixel - center[None, :, :2] for pixel in pixels])
dist2d = (1 / filtersze) ** 2 * np.linalg.norm(dist_xycenter, axis=-1) ** 2
# Min of dist2 is equal to max of Gaussian exp(-0.5 * dist2)
dist2 = np.minimum(dist3d, dist2d)
# dist2 = dist3d
depth_acc = np.sum(homogeneous(s) * T[None, ..., -1], axis=-1)




ValueError: operands could not be broadcast together with shapes (607900,3) (1,6079,3) 

In [64]:
 (1 / filtersze) ** 2 

1.9999999999999996

In [54]:

dist_xycenter = np.hstack([pixel - center[None, :, :2] for pixel in pixels])
dist2d = (1 / filtersze) ** 2 * np.linalg.norm(dist_xycenter, axis=-1) ** 2
dist2d

array([[ 3589.73891366,  3916.54085153,  2631.16579094, ...,
        50206.38488983, 51409.57543689, 50750.8224495 ]])

In [59]:


dist2d

dist3d

array([0.00027445, 0.00026015, 0.00030995, ..., 0.02325993, 0.03146078,
       0.02864169])

In [47]:
# np.minimum(dist2d,dist3d)
dist3d -  np.squeeze(dist2d)

ValueError: operands could not be broadcast together with shapes (607900,) (607900,2) 

In [60]:
np.minimum(dist2d,dist3d)

array([[0.00027445, 0.00026015, 0.00030995, ..., 0.02325993, 0.03146078,
        0.02864169]])

In [ ]:
s = [intersection_point(pixel,T) for pixel in pixels]
s[0].shape # every pixel has an intersection with every center. s[0] - all centers for one pixel, len(s) = len(pixels)

(6079, 2)

In [42]:
bla = s * s
s

array([[-0.01131175,  0.01210363],
       [-0.01136562,  0.01144436],
       [-0.01255127,  0.01234568],
       ...,
       [-0.12215594,  0.09131189],
       [-0.14202954,  0.10624682],
       [-0.13595495,  0.10078661]])

In [10]:
np.column_stack([x, y]).shape


(100, 2)

In [ ]:
# Rasterization
# 1. Generate pixels
W, H = int(intrins[0, -1] * 2), int(intrins[1, -1] * 2)
pix_x, pix_y = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
pix = np.stack([pix_x, pix_y], axis=-1)

# 2. Compute ray splat intersection # Eq.9 and Eq.10
pix_flat = pix.reshape(-1, 1, 2)
x = pix_flat[..., :1]
y = pix_flat[..., 1:]
k = -T[None, ..., 0] + x * T[None, ..., 3]
l = -T[None, ..., 1] + y * T[None, ..., 3]
points = np.cross(k, l, axis=-1)
s = points[..., :2] / points[..., -1:]

In [12]:
    W, H = int(intrins[0, -1] * 2), int(intrins[1, -1] * 2)
    pix_x, pix_y = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
    pix = np.stack([pix_x, pix_y], axis=-1)

    # 2. Compute ray splat intersection # Eq.9 and Eq.10
    pix_flat = pix.reshape(-1, 1, 2)
    x = pix_flat[..., :1]
    y = pix_flat[..., 1:]

x.shape

(79576, 1, 1)

In [ ]:
def surface_splatting(means3D, scales, quats, colors, opacities, intrins, viewmat, projmat):
    # Rasterization setup
    projmat = np.zeros((4, 4))
    projmat[:3, :3] = intrins
    projmat[-1, -2] = 1.0
    projmat = projmat.T
    T, colors, opacities, center, depth, radii = setup(means3D, scales, quats, opacities, colors, viewmat, projmat)

    # Rasterization
    # 1. Generate pixels
    W, H = int(intrins[0, -1] * 2), int(intrins[1, -1] * 2)
    pix_x, pix_y = np.meshgrid(np.arange(W), np.arange(H), indexing='xy')
    pix = np.stack([pix_x, pix_y], axis=-1)

    # 2. Compute ray splat intersection # Eq.9 and Eq.10
    pix_flat = pix.reshape(-1, 1, 2)
    x = pix_flat[..., :1]
    y = pix_flat[..., 1:]
    k = -T[None, ..., 0] + x * T[None, ..., 3]
    l = -T[None, ..., 1] + y * T[None, ..., 3]
    points = np.cross(k, l, axis=-1)
    s = points[..., :2] / points[..., -1:]

    # 3. Add low-pass filter # Eq.11
    # When a point (2D Gaussian) is viewed from a far distance or at a slanted angle,
    # the 2D Gaussian falls between pixels, and no fragment is used to rasterize the Gaussian.
    # Add a low-pass filter to handle aliasing.
    dist3d = np.sum(s * s, axis=-1)
    filtersze = np.sqrt(2) / 2
    dist2d = (1 / filtersze) ** 2 * np.linalg.norm(
        np.concatenate([x, y], axis=-1) - center[None, :, :2], axis=-1
    ) ** 2
    # Min of dist2 is equal to max of Gaussian exp(-0.5 * dist2)
    dist2 = np.minimum(dist3d, dist2d)
    # dist2 = dist3d
    depth_acc = np.sum(homogeneous(s) * T[None, ..., -1], axis=-1)

    # 4. Accumulate 2D Gaussians through alpha blending # Eq.12
    image, depthmap = alpha_blending_with_gaussians(dist2, colors, opacities, depth_acc, H, W)
    return image, depthmap, center, radii, dist2

array([[-19.34314648,   6.08286685, -15.65574901],
       [-46.58113944,   7.48236654, -17.95189715],
       [-18.88483308,   9.37179436, -20.53495226],
       ...,
       [-39.0420906 ,  -3.1487929 ,  -2.10798984],
       [ 60.60087204, -73.58221251, 103.25931432],
       [-28.05912037,   2.82825118, -10.89369607]])

In [ ]:

def alpha_blending(alpha, colors):
    # Calculate cumulative alpha blending weights
    T = np.concatenate([np.ones_like(alpha[-1:]), np.cumprod(1 - alpha, axis=0)[:-1]], axis=0)
    
    # Blend the colors using the alpha values and the cumulative transparency
    image = np.sum(T * alpha * colors, axis=0).reshape(-1, colors.shape[-1])
    
    # Accumulate alpha values for the alpha map
    alphamap = np.sum(T * alpha, axis=0).reshape(-1, 1)
    
    return image, alphamap

def alpha_blending_with_gaussians(dist2, colors, opacities, depth_acc, H, W):
    # Reshape colors and depth accumulation for proper broadcasting
    colors = colors.reshape(-1, 1, colors.shape[-1])
    depth_acc = depth_acc.T[..., None]
    depth_acc = np.repeat(depth_acc, 1, axis=2)  # equivalent to repeating across the 3rd axis

    # Evaluate Gaussians (using cutoff for visualization purposes)
    cutoff = 1**2  # Equivalent to 1 sigma^2
    dist2 = dist2.T  # Transpose to match the expected shape
    gaussians = np.exp(-0.5 * dist2) * (dist2 < cutoff)
    gaussians = gaussians[..., None]  # Add an extra dimension for broadcasting
    alpha = opacities[:, None] * gaussians  # Opacities expanded to match the shape of Gaussians

    # Accumulate Gaussians through alpha blending
    image, _ = alpha_blending(alpha, colors)
    depthmap, alphamap = alpha_blending(alpha, depth_acc)
    
    # Normalize depthmap by the alpha map
    depthmap = depthmap / alphamap
    depthmap = np.nan_to_num(depthmap, nan=0, posinf=0, neginf=0)  # Handle NaN and infinity values
    
    return image.reshape(H, W, -1), depthmap.reshape(H, W, -1)

(6079, 4, 4)

In [11]:
# Surface splatting (2D Gaussian Splatting)
def setup(means3D, scales, quats, opacities, colors, viewmat, projmat):
    rotations = build_scaling_rotation(scales, quats).permute(0,2,1)

    # 1. Viewing transform
    # Eq.4 and Eq.5
    p_view = (means3D @ viewmat[:3,:3]) + viewmat[-1:,:3]
    uv_view = (rotations @ viewmat[:3,:3])
    M = torch.cat([homogeneous_vec(uv_view[:,:2,:]), homogeneous(p_view.unsqueeze(1))], dim=1)

    T = M @ projmat # T stands for (WH)^T in Eq.9
    # 2. Compute AABB
    # Homogneous plane is very useful for both ray-splat intersection and bounding box computation
    # we know how to compute u,v given x,y homogeneous plane already; computing AABB is done by a reverse process.
    # i.e compute the x, y s.t. \|hu^4\| = 1 and \|h_v^4\|=1 (distance of gaussian center to plane in the uv space)
    temp_point = torch.tensor([[1.,1., -1.]]).cuda()
    distance = (temp_point * (T[..., 3] * T[..., 3])).sum(dim=-1, keepdims=True)
    f = (1 / distance) * temp_point
    point_image = torch.cat(
        [(f * T[..., 0] * T[...,3]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,3]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,3]).sum(dim=-1, keepdims=True)], dim=-1)

    half_extend = point_image * point_image - torch.cat(
        [(f * T[..., 0] * T[...,0]).sum(dim=-1, keepdims=True),
        (f * T[..., 1] * T[...,1]).sum(dim=-1, keepdims=True),
        (f * T[..., 2] * T[...,2]).sum(dim=-1, keepdims=True)], dim=-1)

    radii = half_extend.clamp(min=1e-4).sqrt() * 3 # three sigma
    center = point_image

    # 3. Perform Sorting
    depth = p_view[..., 2] # depth is used only for sorting
    index = depth.sort()[1]
    T = T[index]
    colors = colors[index]
    center = center[index]
    depth = depth[index]
    radii = radii[index]
    opacities = opacities[index]
    return T, colors, opacities, center, depth, radii


## 2DGS - Math explained
### Modeling: 
The 2D Gaussian is defined in terms of its local coordinate system, where the $ X$ and $ Y$ axes are scaled according to the Gaussian's shape. The Gaussian is represented in the **world coordinates**, with the **axes** of the Gaussian in world space defined by the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $, and its **scaling factors** by $ s_u $ and $ s_v $. These tangential vectors define the directions of the local coordinate axes in the tangent plane (the object FoR).

The **object plane**, which is the tangent plane to the Gaussian in world space, can be described using the plane equation:
$$
P(u, v) = p_k + s_u \mathbf{t_u} u + s_v \mathbf{t_v} v
$$
where $ p_k $ is the center of the Gaussian in world coordinates, and $ u $ and $ v $ represent local coordinates on the tangent plane. This equation expresses the position of any point in the tangent plane in terms of the world coordinates, as modified by the scaling along the tangential axes $ \mathbf{t_u} $ and $ \mathbf{t_v} $.

To transform the local coordinates in the tangent plane (object frame of reference) to world coordinates, we define the following transformation matrix $ H $, which encodes the **scaling**, **rotation**, and **translation** from the object space to the world space:
$$
H = \begin{pmatrix}
s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 & p_k \\
0 & 0 & 0 & 1
\end{pmatrix}
= \begin{pmatrix}
R & p_k \\
0 & 1
\end{pmatrix}
$$
where:
- $ \mathbf{t_u} = \begin{pmatrix} t_{u_x} \\ t_{u_y} \\ t_{u_z} \end{pmatrix} $ and $ \mathbf{t_v} = \begin{pmatrix} t_{v_x} \\ t_{v_y} \\ t_{v_z} \end{pmatrix} $ are the 3D tangential vectors (defining the axes of the tangent plane in world coordinates),
- $ p_k = \begin{pmatrix} p_{k_x} \\ p_{k_y} \\ p_{k_z} \end{pmatrix} $ is the 3D position of the Gaussian center in world coordinates, and
- $ R $ is the 3x3 rotation matrix that describes the orientation of the tangent plane in world space.

The matrix $ H $ can be interpreted in two parts:
- The first part, $ \begin{pmatrix} s_u \mathbf{t_u} & s_v \mathbf{t_v} & 0 \end{pmatrix} $, represents the scaling and rotation of the axes in the world coordinates. It scales the tangential vectors $ \mathbf{t_u} $ and $ \mathbf{t_v} $ by $ s_u $ and $ s_v $, respectively, and applies any rotational transformation.
- The second part, $ p_k $, represents the **translation** of the Gaussian's center in world coordinates, which shifts the origin of the local tangent plane to the desired world position.

The matrix can also be interpreted in terms of the rotation matrix $ R $ (which describes the orientation of the tangent plane in world space) and the translation vector $ p_k $. The transformation from local coordinates $ (u, v) $ to world coordinates $ (x, y, z) $ is thus achieved through the multiplication of $ H $ by the local coordinate vector:
$$
\begin{pmatrix}
x \\
y \\
z \\
1
\end{pmatrix}
= H \begin{pmatrix}
u \\
v \\
1 \\
1
\end{pmatrix}
$$

This transformation allows us to map a point in the tangent plane (object space) to the corresponding point in the world space, accounting for both the Gaussian's shape (through the scaling $ s_u, s_v $) and its orientation (through the rotation $ R $).

In summary, the matrix $ H $ serves as a **homogeneous transformation** that encapsulates both the geometric properties (scaling and rotation) and the positioning (translation) of the 2D Gaussian in world space.

for every u,v on the tangent plane we can calculate the gaussian power: 
$$ G(u) = \exp\left( -\frac{u^2 + v^2}{2} \right) \tag{6} $$

* $ \mathbf{t_u} $ , $ \mathbf{t_v} , $ $\mathbf{s_u} $ and $\mathbf{s_v}$ are learnable parameters.
* each gaussian is defined by the opacity $\alpha$ and a view dependent color.

### Splatting


The goal of this process is to find the intersection between a ray, originating from a pixel in the image, and the tangent plane of a 2D Gaussian, and then evaluate the power of the Gaussian at the intersection point. Below are the detailed steps.

Step 1: Defining the Image Ray 

We begin by defining the ray from a pixel in the image using two orthogonal planes. The pixel location $(x, y) $ in the image space can be used to define two planes:


* The x-plane (yz): a plane defined by a normal vector $ \mathbf{n}_x = (-1, 0, 0) $ and an offset $ x $. The 4D homogeneous form is $ h_x = (-1, 0, 0, x) $.
* The y-plane (xz): a plane defined by a normal vector $ \mathbf{n}_y = (0, -1, 0) $ and an offset $ y $. The 4D homogeneous form is $ h_y = (0, -1, 0, y) $.


These planes intersect at a ray in 3D space that is represented as the line of intersection between the two planes.

Step 2: Transforming the Planes to the Tangent Plane

Next, we transform the planes from the image space into the local coordinates of the 2D Gaussian primitive (the tangent plane). Using the transformation matrix $ M = (WH)^{-1} $, we apply the inverse transpose to the planes (W is the prespective projection matrix (transformation from camera to screen space), H is a transformation from tangant plane to camera space):

multiplying W (camera to screen) by H (tangent plane to camera) will transform a coordinate from tangent plane to camera to screen. Transposing it will map from screen space to tangent plane 

$$
h_u = (WH)^\top h_x \\
h_v = (WH)^\top h_y
$$
* those are the planes rotated from image space to tangent space

Step 3: Solving for the Intersection

The next step is to solve for the intersection point $(u, v) $ of the ray with the transformed tangent plane. We do this by solving the following system of equations:

$$
h_u \cdot (u, v, 1, 1)^\top = 0 \\
h_v \cdot (u, v, 1, 1)^\top = 0
$$

Expanding these equations gives us:


$$
    h_1^u u + h_2^u v + h_3^u + h_4^u = 0 \\
    h_1^v u + h_2^v v + h_3^v + h_4^v = 0
$$


The solution for $ u $ and $ v $ is obtained by solving this system of equations. This leads to the following expressions for the coordinates $ u(x) $ and $ v(x) $:

$$
u = \frac{h_2^u h_4^v - h_4^u h_2^v}{h_1^u h_2^v - h_2^u h_1^v} \\
v = \frac{h_4^u h_1^v - h_1^u h_4^v}{h_1^u h_2^v - h_2^u h_1^v}
$$

Where $ h_i^u $ and $ h_i^v $ are the homogeneous parameters of the transformed planes.

Step 4: Evaluating the Gaussian at the Intersection

Once we have the coordinates $ (u, v) $, we can evaluate the 2D Gaussian function at the intersection point. The Gaussian function is typically of the form:

$$
G(u, v) = \exp\left( -\frac{u^2 + v^2}{2} \right)
$$

This represents the Gaussian value at the point $ (u, v) $ on the tangent plane.


Once the values of $ u(x) $ and $ v(x) $ are found, we can compute the depth of the intersection point using the following equation:
$$
x = (x_z, y_z, z, z)^\top = W P(u, v) = W H (u, v, 1, 1)^\top
$$
Here, $ W $ is the camera projection matrix, and $ H $ is the transformation matrix from the tangent plane to world coordinates. The last component of the resulting vector $ x = (xz, yz, z, z)^\top $ gives the depth $ z $ of the intersection point (the third element, need to make sure). 



#### Screen space
this transformation returns a 3d coordinate in screen space. 
, where rendering occures. the projection matrix transformes x,y,z to the screen space
* x axis spans from 0 to -1 (width)
* y axis spans from 0 to -1 (hight)
* Z_{NDC} represents the depth (need to check that)
from NDC we can map to pixels

#### Transforming planes
* A point in 3D homogeneous coordinates is represented as $ (x,y,z,w)
* a plane consist of both a normal and an offset - in 3D homogeneous coordinates it is represented as $(a,b,c,d)
where $(a,b,c) is the planes normal vector and $d$ is its offset. 

The plane equation : $ ax + by + cz + dw = 0 $
which means that that dot product of the plane parameters $ (a,b,c,d)$ and a point $(x,y,z,w)$ is zero for a point lying on the plane. 

** Transforming Point on a Plane **
we Define matrix $M$ - a transformation matrix. 
tansforming a point on the plain using $M$: $p'=M\cdot p$
after transforming it, the plane equation should still hold: $h'p' = 0$ were $h', p'$ are the transformed plane and point parameters.

Because $p' = M\cdot p$ we get that $h'(M\cdot p) = 0 $ and since $ h' \cdot(M\cdotp p)=M^T \cdot h' \cdot p $ we can write 
$$
M^T \cdot h' \cdot p = h\cdot p \\
M^T \cdot h' = h
$$

and from that we show that the plane $h'$ remains valid after transformation. 
